In [43]:
# Imports
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
train = pd.read_csv("data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [9]:
# Sample Review
print train['review'][2]

"The film starts with a manager (Nicholas Bell) giving welcome investors (Robert Carradine) to Primal Park . A secret project mutating a primal animal using fossilized DNA, like ¨Jurassik Park¨, and some scientists resurrect one of nature's most fearsome predators, the Sabretooth tiger or Smilodon . Scientific ambition turns deadly, however, and when the high voltage fence is opened the creature escape and begins savagely stalking its prey - the human visitors , tourists and scientific.Meanwhile some youngsters enter in the restricted area of the security center and are attacked by a pack of large pre-historical animals which are deadlier and bigger . In addition , a security agent (Stacy Haiduk) and her mate (Brian Wimmer) fight hardly against the carnivorous Smilodons. The Sabretooths, themselves , of course, are the real star stars and they are astounding terrifyingly though not convincing. The giant animals savagely are stalking its prey and the group run afoul and fight against on

In [41]:
# Preprocessing
def clean_review (review):
    review_text = BeautifulSoup(review).get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stop_words = set(stopwords.words("english"))
    filtered_words = [w for w in words if not w in stop_words]
    # Lemmatizing
    wordnet_lemmatizer = WordNetLemmatizer()
    word_list = [wordnet_lemmatizer.lemmatize(w) for w in filtered_words]
    
    return (" ".join(word_list))
    

In [42]:
num_reviews = train['review'].size
clean_reviews = []
for i in xrange(0, num_reviews):
    if ((i+1)%100 == 0):
        print i+1
    clean_reviews.append(clean_review(train['review'][i]))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [45]:
# Creating Bag of Words Model
vectorizer = CountVectorizer(analyzer = "word",
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 5000)
train_data_features = vectorizer.fit_transform(clean_reviews).toarray()

In [48]:
# One feature for each vocab word
train_data_features.shape

(25000, 5000)

In [50]:
# Words in our vocabulary
vocab = vectorizer.get_feature_names()
print vocab

[u'abandoned', u'abc', u'ability', u'able', u'abraham', u'abrupt', u'absence', u'absent', u'absolute', u'absolutely', u'absurd', u'absurdity', u'abuse', u'abused', u'abusive', u'abysmal', u'academy', u'accent', u'accept', u'acceptable', u'accepted', u'accepts', u'access', u'accident', u'accidentally', u'acclaimed', u'accompanied', u'accomplish', u'accomplished', u'according', u'account', u'accuracy', u'accurate', u'accused', u'ace', u'achieve', u'achieved', u'achievement', u'acid', u'across', u'act', u'acted', u'acting', u'action', u'active', u'activity', u'actor', u'actress', u'actual', u'actually', u'ad', u'adam', u'adaptation', u'adapted', u'add', u'added', u'addict', u'addicted', u'addiction', u'adding', u'addition', u'additional', u'address', u'adequate', u'admirable', u'admire', u'admit', u'admittedly', u'adolescent', u'adopted', u'adorable', u'adult', u'advance', u'advanced', u'advantage', u'adventure', u'advertising', u'advice', u'advise', u'affair', u'affect', u'affected', u'a

In [52]:
# Random Forrest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(train_data_features, train['sentiment'])

In [53]:
# Creating Submission
test = pd.read_csv("data/testData.tsv", header=0, delimiter='\t', quoting=3)
num_test_reviews = len(test["review"])
clean_test_reviews = []

for i in xrange(0, num_test_reviews):
    if ((i+1) % 500 == 0):
        print i+1
    test_review = clean_review(test['review'][i])
    clean_test_reviews.append(test_review)
    
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000


IOError: [Errno 2] No such file or directory: 'result/bag_of_words_rf.csv'

In [55]:
output.to_csv("bag_of_words_rf.csv", index=False, quoting=3)